In [77]:
import random
from difflib import SequenceMatcher
from konlpy.tag import Okt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Spotify 인증
SPOTIPY_CLIENT_ID = 'e0b9304753a04c0c90c887066cd702ce'
SPOTIPY_CLIENT_SECRET = '79e2ff5799794bc0bd672ae97388aaaa'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET
))

# 감정별 대표 단어 사전
emotion_to_situation = {
    '기쁨': ['행복', '설레는', '썸', '연애', '달달한', '산책', '에너지'],
    '슬픔': ['슬픔', '밤', '고통', '눈물', '이별', '외로움', '우울', '고독', '절망', '답답', '막막', '감성'],
    '분노': ['분노', '짜증', '폭발', '스트레스', '싸움', '격렬', '부셔', '빡칠때']
}

# 불용어 제거용
stopwords = {"오늘", "진짜", "정말", "내일", "지금", "그냥"}

# 형태소 분석기
okt = Okt()

def extract_keywords_pos(text):
    tokens = okt.pos(text, stem=True)
    return [word for word, tag in tokens if tag in ['Noun', 'Adjective'] and word not in stopwords]

def get_best_matching_emotion_word(user_input, emotion):
    user_keywords = extract_keywords_pos(user_input)
    best_word = None
    best_score = 0
    candidates = emotion_to_situation.get(emotion, [])

    for user_word in user_keywords:
        for emo_word in candidates:
            score = SequenceMatcher(None, user_word, emo_word).ratio()
            if score > best_score:
                best_score = score
                best_word = emo_word

    return best_word if best_word else random.choice(candidates)

def recommend_music(user_input, emotion):
    # 1. 대표 감정 단어 추출
    query_for_search = get_best_matching_emotion_word(user_input, emotion)
    query_for_similarity = query_for_search

    print(f"[감정 기반 검색 쿼리] {query_for_search}")

    # 2. Spotify playlist 검색
    try:
        results = sp.search(q=query_for_search, type='playlist', limit=50, market='KR')
        playlists = results.get('playlists', {}).get('items', [])
    except Exception as e:
        print(f"❗ Spotify 검색 오류: {e}")
        return None

    if not playlists:
        print("❗ 검색된 플레이리스트가 없습니다.")
        return None

    # 3. 제목 유사도 기반 추천
    titles = [p.get('name', '') + ' ' + p.get('description', '') for p in playlists if p]
    if not titles:
        print("❗ 플레이리스트 제목이 없습니다.")
        return None

    try:
        vectorizer = TfidfVectorizer().fit(titles + [query_for_similarity])
        vectors = vectorizer.transform(titles + [query_for_similarity])
        sims = cosine_similarity(vectors[-1], vectors[:-1]).flatten()
    except Exception as e:
        print(f"❗ 유사도 계산 오류: {e}")
        return None

    top5_idx = sims.argsort()[-5:][::-1]
    valid_indices = [i for i in top5_idx if playlists[i] is not None]

    if not valid_indices:
        print("❗ 유효한 추천 후보가 없습니다.")
        return None

    best_idx = random.choice(valid_indices)
    playlist = playlists[best_idx]

    playlist_id = playlist.get('id')
    playlist_name = playlist.get('name')
    playlist_url = playlist.get('external_urls', {}).get('spotify')

    # 4. 트랙 정보 수집
    try:
        tracks_data = sp.playlist_tracks(playlist_id, limit=50, market="KR")
        all_tracks = tracks_data.get('items', [])
        selected_tracks = random.sample(all_tracks, k=min(3, len(all_tracks)))
    except Exception as e:
        print(f"❗ 트랙 정보 수집 실패: {e}")
        selected_tracks = []

    songs = []
    for t in selected_tracks:
        track = t.get('track')
        if track:
            songs.append({
                'name': track.get('name'),
                'artist': track.get('artists', [{}])[0].get('name', 'Unknown'),
                'url': track.get('external_urls', {}).get('spotify')
            })

    return {
        'emotion': emotion,
        'query': query_for_search,
        'playlist': playlist_name,
        'playlist_url': playlist_url,
        'songs': songs
    }

In [96]:
import random
from konlpy.tag import Okt
from sentence_transformers import SentenceTransformer, util
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Spotify 인증
SPOTIPY_CLIENT_ID = 'e0b9304753a04c0c90c887066cd702ce'
SPOTIPY_CLIENT_SECRET = '79e2ff5799794bc0bd672ae97388aaaa'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET
))

# 감정별 대표 단어 사전
emotion_to_situation = {
    '기쁨': ['행복', '설레는', '썸', '연애', '달달한', '산책', '에너지'],
    '슬픔': ['슬픔', '밤', '고통', '눈물', '이별', '외로움', '우울', '고독', '절망', '답답', '막막', '감성'],
    '분노': ['분노', '짜증', '폭발', '스트레스', '싸움', '격렬', '부셔', '빡칠때']
}

# Sentence-BERT 모델 로드 (최초 1회)
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def get_best_matching_emotion_word(user_input, emotion):
    candidates = emotion_to_situation.get(emotion, [])
    sentences = [user_input] + candidates

    embeddings = model.encode(sentences, convert_to_tensor=True)
    user_embedding = embeddings[0]
    candidate_embeddings = embeddings[1:]

    cosine_scores = util.cos_sim(user_embedding, candidate_embeddings)[0]
    best_idx = cosine_scores.argmax().item()

    return candidates[best_idx]

def recommend_music(user_input, emotion):
    query_for_search = get_best_matching_emotion_word(user_input, emotion)
    print(f"[감정 기반 검색 쿼리] {query_for_search}")

    try:
        results = sp.search(q=query_for_search, type='playlist', limit=50, market='KR')
        playlists = results.get('playlists', {}).get('items', [])
    except Exception as e:
        print(f"❗ Spotify 검색 오류: {e}")
        return None

    if not playlists:
        print("❗ 검색된 플레이리스트가 없습니다.")
        return None

    playlist = random.choice(playlists)
    playlist_id = playlist.get('id')
    playlist_name = playlist.get('name')
    playlist_url = playlist.get('external_urls', {}).get('spotify')

    try:
        tracks_data = sp.playlist_tracks(playlist_id, limit=50, market="KR")
        all_tracks = tracks_data.get('items', [])
        selected_tracks = random.sample(all_tracks, k=min(3, len(all_tracks)))
    except Exception as e:
        print(f"❗ 트랙 정보 수집 실패: {e}")
        selected_tracks = []

    songs = []
    for t in selected_tracks:
        track = t.get('track')
        if track:
            songs.append({
                'name': track.get('name'),
                'artist': track.get('artists', [{}])[0].get('name', 'Unknown'),
                'url': track.get('external_urls', {}).get('spotify')
            })

    return {
        'emotion': emotion,
        'query': query_for_search,
        'playlist': playlist_name,
        'playlist_url': playlist_url,
        'songs': songs
    }

In [97]:
if __name__ == "__main__":
    user_input = "그 얘기만 하면 속에서 분노가 치밀어 올라."
    emotion = "분노"

    result = recommend_music(user_input, emotion)

    if result:
        print(f"\n감정: {result['emotion']}")
        print(f"추천 플레이리스트: {result['playlist']}")
        print(f"링크: {result['playlist_url']}\n")
        print("추천 곡 리스트:")
        for song in result['songs']:
            print(f"- {song['name']} - {song['artist']}\n  🔗 {song['url']}")
    else:
        print("추천 결과를 가져오지 못했습니다.")

[감정 기반 검색 쿼리] 분노

감정: 분노
추천 플레이리스트: 분노의질주!!
링크: https://open.spotify.com/playlist/0ZND4zpYcRk0CpMraZOvAD

추천 곡 리스트:
- See You Again (feat. Charlie Puth) - Wiz Khalifa
  🔗 https://open.spotify.com/track/4Aep3WGBQlpbKXkW7kfqcU
- My City - 24kGoldn
  🔗 https://open.spotify.com/track/2g8OLnBVj5bWD4oY2ZIgho
- Viva La Vida - Coldplay
  🔗 https://open.spotify.com/track/1mea3bSkSGXuIRvnydlB5b


In [1]:
import random
from konlpy.tag import Okt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Spotify 인증
SPOTIPY_CLIENT_ID = 'e0b9304753a04c0c90c887066cd702ce'
SPOTIPY_CLIENT_SECRET = '79e2ff5799794bc0bd672ae97388aaaa'

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET
))

# 감정별 대표 단어 사전
emotion_to_situation = {
    '기쁨': ['행복', '설레는', '썸', '연애', '달달한', '산책', '에너지'],
    '슬픔': ['슬픔', '밤', '고통', '눈물', '이별', '외로움', '우울', '고독', '절망', '답답', '막막', '감성'],
    '분노': ['분노', '짜증', '폭발', '스트레스', '미친', '싸움', '격렬', '부셔', '빡칠때']
}

# 불용어 제거용
stopwords = {"오늘", "진짜", "정말", "내일", "지금", "그냥"}

# 형태소 분석기
okt = Okt()

# SBERT 로딩
print("[INFO] Loading Sentence-BERT model...")
# model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
# model = SentenceTransformer('jhgan/ko-sroberta-multitask')
model = SentenceTransformer('smartmind/roberta-ko-small-tsdae')

def extract_keywords_pos(text):
    tokens = okt.pos(text, stem=True)
    return [word for word, tag in tokens if tag in ['Noun', 'Adjective'] and word not in stopwords]

def get_best_matching_emotion_word(user_input, emotion):
    user_keywords = extract_keywords_pos(user_input)
    candidates = emotion_to_situation.get(emotion, [])
    if not user_keywords:
        return random.choice(candidates)

    user_sent = " ".join(user_keywords)
    queries = [user_sent] + candidates
    embeddings = model.encode(queries)
    sims = cosine_similarity([embeddings[0]], embeddings[1:])[0]
    best_idx = sims.argmax()
    return candidates[best_idx]

def recommend_music(user_input, emotion):
    # 1. 대표 감정 단어 추출 (SBERT 유사도 기반)
    query_for_search = get_best_matching_emotion_word(user_input, emotion)
    query_for_similarity = user_input

    print(f"[감정 기반 검색 쿼리] {query_for_search}")

    # 2. Spotify playlist 검색
    results = sp.search(q=query_for_search, type='playlist', limit=50, market='KR')
    raw_playlists = results.get('playlists', {}).get('items', [])

    # 3. None 또는 필드 누락된 항목 필터링
    playlists = [
        p for p in raw_playlists
        if p and isinstance(p, dict) and 'name' in p and 'id' in p and 'external_urls' in p
    ]
    if not playlists:
        return None

    # 4. 제목 + 설명 임베딩 생성
    playlist_texts = [p['name'] + ' ' + p.get('description', '') for p in playlists]
    playlist_embeddings = model.encode(playlist_texts)
    query_embedding = model.encode([query_for_similarity])[0]

    sims = cosine_similarity([query_embedding], playlist_embeddings).flatten()
    top5_idx = sims.argsort()[-5:][::-1]
    best_idx = random.choice(top5_idx)
    playlist = playlists[best_idx]

    if not playlist or not isinstance(playlist, dict):
        return None

    playlist_id = playlist['id']
    playlist_name = playlist['name']
    playlist_url = playlist['external_urls']['spotify']

    try:
        tracks_data = sp.playlist_tracks(playlist_id, limit=50, market="KR")
        all_tracks = tracks_data['items']
        selected_tracks = random.sample(all_tracks, k=min(3, len(all_tracks)))
    except:
        selected_tracks = []

    songs = []
    for t in selected_tracks:
        if not t or not t.get('track'):
            continue
        track = t['track']
        if 'name' not in track or 'artists' not in track or 'external_urls' not in track:
            continue
        songs.append({
            'name': track['name'],
            'artist': track['artists'][0]['name'],
            'url': track['external_urls']['spotify']
        })

    return {
        'emotion': emotion,
        'query': query_for_search,
        'playlist': playlist_name,
        'playlist_url': playlist_url,
        'songs': songs
    }

/Users/choiinjoon/miniconda3/envs/ia3-clean/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Loading Sentence-BERT model...


In [26]:
if __name__ == "__main__":
    user_input = "비 오니까 괜히 눈물 나는 노래 듣고 싶어"
    emotion = "슬픔"

    result = recommend_music(user_input, emotion)

    if result:
        print(f"\n감정: {result['emotion']}")
        print(f"추천 플레이리스트: {result['playlist']}")
        print(f"링크: {result['playlist_url']}\n")
        print("추천 곡 리스트:")
        for song in result['songs']:
            print(f"- {song['name']} - {song['artist']}\n / 🔗 {song['url']}")
    else:
        print("추천 결과를 가져오지 못했습니다.")

[감정 기반 검색 쿼리] 눈물

감정: 슬픔
추천 플레이리스트: 남자라면 울어버리는 낭만100프로 눈물 광광 락발라드
링크: https://open.spotify.com/playlist/4p3fD9lZ1iBjGGPZtYbUmk

추천 곡 리스트:
- 까만안경 (FEAT. DAY LIGHT) - ERU
 / 🔗 https://open.spotify.com/track/2Lr3T69cweqvRihO8SIoSS
- Endless - 플라워
 / 🔗 https://open.spotify.com/track/29IGdJQsLN56BEaUzh7YOS
- 내일 할 일 (with Sung Si Kyung) - Yoon Jong Shin
 / 🔗 https://open.spotify.com/track/5NLnOyDrGmRWwavktF6Ens


In [22]:
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials

SPOTIPY_CLIENT_ID = 'e0b9304753a04c0c90c887066cd702ce'
SPOTIPY_CLIENT_SECRET = '79e2ff5799794bc0bd672ae97388aaaa'

sp = Spotify(auth_manager=SpotifyClientCredentials(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET
))

emotion_to_keywords = {
    '기쁨': ['행복해지는', '기쁨', '설렘', '연애', '기분좋은'],
    '슬픔': ['이별노래', '우울함', '외로울 때', '밤에', '슬플 때'],
    '분노': ['짜증', '화날 때', '스트레스', '미친', '빡칠때']
}

collected_titles = {}

for emotion, keywords in emotion_to_keywords.items():
    collected_titles[emotion] = []
    print(f"\n🔍 감정: {emotion}")
    
    for keyword in keywords:
        print(f"  - 검색어: {keyword}", end=" ")
        try:
            results = sp.search(q=keyword, type='playlist', limit=20, market = 'KR')
            items = results.get('playlists', {}).get('items', [])
            
            if items is None:
                raise ValueError("검색 결과가 없습니다.")

            count = 0
            for playlist in items:
                name = playlist.get('name')
                if name:
                    collected_titles[emotion].append(name)
                    count += 1

            print(f"✅ 수집: {count}개")
        
        except Exception as e:
            print(f"⚠️ 에러 발생: {e}")

    print(f"🎧 총 수집된 {emotion} 플레이리스트 제목: {len(collected_titles[emotion])}개")


🔍 감정: 기쁨
  - 검색어: 행복해지는 ✅ 수집: 14개
  - 검색어: 기쁨 ✅ 수집: 17개
  - 검색어: 설렘 ✅ 수집: 19개
  - 검색어: 연애 ✅ 수집: 20개
  - 검색어: 기분좋은 ✅ 수집: 19개
🎧 총 수집된 기쁨 플레이리스트 제목: 89개

🔍 감정: 슬픔
  - 검색어: 이별노래 ✅ 수집: 20개
  - 검색어: 우울함 ✅ 수집: 11개
  - 검색어: 외로울 때 ✅ 수집: 8개
  - 검색어: 밤에 ✅ 수집: 20개
  - 검색어: 슬플 때 ✅ 수집: 20개
🎧 총 수집된 슬픔 플레이리스트 제목: 79개

🔍 감정: 분노
  - 검색어: 짜증 ✅ 수집: 16개
  - 검색어: 화날 때 ✅ 수집: 19개
  - 검색어: 스트레스 ✅ 수집: 18개
  - 검색어: 미친 ✅ 수집: 20개
  - 검색어: 빡칠때 ✅ 수집: 17개
🎧 총 수집된 분노 플레이리스트 제목: 90개


In [23]:
# 감정별 플레이리스트 제목 출력 함수
def print_collected_titles(collected_titles):
    for emotion, titles in collected_titles.items():
        print(f"\n🎧 [{emotion}] 플레이리스트 제목 목록 ({len(titles)}개):")
        for i, title in enumerate(titles, 1):
            print(f"{i:2d}. {title}")

# 예시 호출
print_collected_titles(collected_titles)


🎧 [기쁨] 플레이리스트 제목 목록 (89개):
 1. 기분 좋아지는  행복한 노래
 2. 출근길이 조금이나마 행복해지는 팝송모음🎈 ※반박시 바로 사과
 3. 듣기만해도 행복해지는 플레이리스트
 4. 들으면 행복해지는 팝
 5. 틀어놓으면 그냥 기분이 좋아지는 노래모음 / 산뜻하고 행복한노래 국내음악 
 6. 기분이 좋아지는 진짜 좋은 상쾌한 팝송
 7. 행복해지는노래
 8. 행복해지는 올드팝
 9. 2024 행복해지는 음악들
10. 듣기만 해도 행복해지는 노래들🥰
11. 듣기만 해도 행복해지는 노래들🥰
12. 출근길을 활기로 가득 채워줄 플리
13. 행복해지는 팝송
14. 아무 생각 없이 듣는 플리
15. all was well_마음이 죽은 사람들에게, 가사없는 음악
16. 내 기쁨은 너가 벤틀리를 끄는 거야
17. 일의 기쁨과 슬픔 Vol.2
18. 내 기쁨은 네가 벤틀리를 끄는 거야
19. 기쁨이
20. 좆뱅이 플레이리스트
21. 기쁨
22. 기쁨이
23. 일의 기쁨과 슬픔 5
24. 일의 기쁨과 슬픔  The whole volume
25. 🌌 무한 우주에 순간의 빛일지라도 : 9시간
26. 나의 기쁨 나의 노래
27. 내 기쁨은 늘 질투가 되고 슬픔은 항상 약점이 돼
28. 슬픔은 늘 떼로 오고 기쁨은 스쳐 간다
29. 내 기쁨은 너가 벤틀리를 끄는 거야
30. 기쁨
31. 내 기쁨은 너가 벤틀리를 끄는 거야
32. [ 𝐏𝐥𝐚𝐲𝐥𝐢𝐬𝐭  ] 설레는 사랑노래
33. [JPOP] 청량+설렘 한스푼 첨가한 청춘 애니 주제곡 같은 제이팝/제이락 모음🌻  
34. 설레는 달달한 노래💗
35. 몽글몽글 설렘
36.  【Playlist】 너랑 썸탈때 듣는 노래
37. 설렘 지수 200%  연애세포를 깨워줄, 달달한 팝송
38. 설렘주의
39. 듣기만 해도 설렘 충전되는 썸 탈 때 들으면 더 달달한 사랑 노래모음
40. 달달한 짝사랑 플레이리스트❤️
41. by the way_한 여름의 헌책방 직원, 아주 조용한 음악
42. 설렘지수 200% 연애 세포 꺠워주는 달

In [21]:
from konlpy.tag import Okt
from collections import Counter, defaultdict

okt = Okt()
stopwords = {"노래", "플레이리스트", "모음", "듣는", "듣기", "좋은", "때", "음악", "이런", "이럴", "그냥", "다", "좀", "너무"}

def extract_keywords_from_titles(collected_titles, top_n=10):
    emotion_keywords = {}

    for emotion, titles in collected_titles.items():
        all_words = []
        for title in titles:
            tokens = okt.pos(title, stem=True)
            words = [word for word, tag in tokens if tag in ("Noun", "Adjective") and word not in stopwords]
            all_words.extend(words)

        # 상위 키워드 추출
        counter = Counter(all_words)
        emotion_keywords[emotion] = counter.most_common(top_n)

    return emotion_keywords

# 키워드 추출
keywords_by_emotion = extract_keywords_from_titles(collected_titles, top_n=10)

# 결과 출력
for emo, keywords in keywords_by_emotion.items():
    print(f"\n🔍 [{emo}] 상위 키워드:")
    for word, count in keywords:
        print(f"  - {word}: {count}회")


🔍 [기쁨] 상위 키워드:
  - 설레다: 21회
  - 기분: 19회
  - 좋다: 16회
  - 행복하다: 14회
  - 팝송: 14회
  - 기쁨: 14회
  - 연애: 11회
  - 달달: 8회
  - 세포: 6회
  - 리스트: 5회

🔍 [슬픔] 상위 키워드:
  - 슬프다: 23회
  - 이별: 15회
  - 밤: 15회
  - 감성: 10회
  - 좋다: 7회
  - 우울하다: 6회
  - 우울함: 6회
  - 외롭다: 5회
  - 팝송: 5회
  - 발라드: 4회

🔍 [분노] 상위 키워드:
  - 미치다: 17회
  - 빡치다: 14회
  - 스트레스: 13회
  - 짜증나다: 9회
  - 팝송: 8회
  - 개: 7회
  - 부수: 6회
  - 수면: 6회
  - 플리: 5회
  - 기분: 4회
